##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Safety Quickstart

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/safety_python.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/tutorials/safety_python.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

The Gemini API is careful not to return inapropriate responses. What's apropriate can change depending on your use case or audience, so the safety settings are configurable.


## Prerequisites

You can run this quickstart in [Google Colab](https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/python_quickstart.ipynb), which runs this notebook directly in the browser and does not require additional environment configuration.

Alternatively, to complete this quickstart locally, ensure that your development environment meets the following requirements:

-  Python 3.9+
-  An installation of `jupyter` to run the notebook.

## Setup

### Install the Python SDK

The Python SDK for the Gemini API, is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip:

In [2]:
!pip install -q -U google-generativeai

### Import packages

Import the necessary packages.

In [3]:
import google.generativeai as genai

import google.ai.generativelanguage as glm

### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`

In [4]:
import os
try:
    from google.colab import userdata
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)    
except ImportError:
    pass


## Prompt Feedback

The result returned by the `Model.generate_content` method is a `genai.GenerateContentResponse` (a wrapper around a `glm.GenerateContentResponse` object).

In [ ]:
model = genai.GenerativeModel('gemini-1.0-pro')

bad_prompt =     # Put your bad prompt here
response = model.generate_content(bad_prompt)

This response object gives you safety feedback in two ways, first the `prompt_feedback` (a `glm.PromptFeedback`) attribute contains a list of `glm.SafetyRatings` for the input prompt, and an optional `block_reason` (a `glm.PromptFeedback.BlockReason`): 

In [21]:
bool(response.prompt_feedback.block_reason)

True

In [22]:
response.prompt_feedback

block_reason: SAFETY
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: HIGH
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: HIGH
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}

If the prompt is blocked because of the safety ratings, you will not get any candidates in the response:

In [23]:
response.candidates

[]

### Safety settings

Adjust the safety settings and the prompt is no longer blocked:

In [ ]:
response = model.generate_content(
    bad_prompt,
    safety_settings={
        'HATE': 'BLOCK_NONE',
        'HARASSMENT': 'BLOCK_NONE',
    })

While the SDK makes it simple and flexible to specify the safety settings (above), API actually expects a list of `glm.SafetySetting` objects (or their JSON representations). The ocode below is an equivalent to the code above:

In [33]:
response = model.generate_content(
    bad_prompt,
    safety_settings=[
        glm.SafetySetting(category=glm.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
                          threshold=glm.SafetySetting.HarmBlockThreshold.BLOCK_NONE),
        glm.SafetySetting(category=glm.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
                          threshold=glm.SafetySetting.HarmBlockThreshold.BLOCK_NONE)])

With the new settings, the `prompt_feedback` doesn't changed, except that the `blocked_reason` is no longer set.

In [26]:
bool(response.prompt_feedback.block_reason)

False

And a candidate response is returned.

In [27]:
len(response.candidates)

1

### Candidate ratings

For a prompt that is not blocked, the response object contains a list of `glm.Candidate` objects (just 1 for now). Each candidate includes a `finish_reason` (from `glm.Candidate.FinishReason`): 

In [28]:
candidate = response.candidates[0]
candidate.finish_reason

<FinishReason.STOP: 1>

`FinishReason.STOP` means that the model finished it's output normally. If a candidate's finish reason is `SAFETY` it's becauase the candidate's `safety_ratings` exceeded the the request's `safety_settings` threshold. This response was rated `NEGLIGIBLE` harm probability: 

In [29]:
candidate.safety_ratings

[category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: NEGLIGIBLE
]

The `safety_settings`  